In [1]:
!pip install optuna

In [1]:
import random
import os

import numpy as np
import pandas as pd

from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import missingno

# device 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print (device)

seed = 42 # seed 값 설정
random.seed(seed) # 파이썬 난수 생성기
os.environ['PYTHONHASHSEED'] = str(seed) # 해시 시크릿값 고정
np.random.seed(seed) # 넘파이 난수 생성기

torch.manual_seed(seed) # 파이토치 CPU 난수 생성기
torch.backends.cudnn.deterministic = True # 확정적 연산 사용 설정
torch.backends.cudnn.benchmark = False   # 벤치마크 기능 사용 해제
torch.backends.cudnn.enabled = False        # cudnn 기능 사용 해제

if device == 'cuda':
    torch.cuda.manual_seed(seed) # 파이토치 GPU 난수 생성기
    torch.cuda.manual_seed_all(seed) # 파이토치 멀티 GPU 난수 생성기

cpu


In [2]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [3]:
import pandas as pd
import numpy as np
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors

def calculate_metabolic_stability_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    
    if mol is None:
        return [None] * 11  # 제거한 특성의 수만큼 None 값 반환

    logP = Descriptors.MolLogP(mol)
    apka = Descriptors.MolWt(mol)
    num_rotatable_bonds = Descriptors.NumRotatableBonds(mol)
    num_hydrogen_acceptors = Descriptors.NumHAcceptors(mol)
    num_hydrogen_donors = Descriptors.NumHDonors(mol)
    morgan_fingerprint = AllChem.GetHashedMorganFingerprint(mol, 6, nBits=4096)
    morgan_array = np.zeros((1,), dtype=np.int8)
    DataStructs.ConvertToNumpyArray(morgan_fingerprint, morgan_array)
    
    tpsa = Descriptors.TPSA(mol)
    j_index = Descriptors.BalabanJ(mol)
    num_rings = len(Chem.GetSymmSSSR(mol))
    num_aromatic_rings = Descriptors.NumAromaticRings(mol)
    
    molar_refractivity = Descriptors.MolMR(mol)
    
    return logP, apka, num_rotatable_bonds, num_hydrogen_acceptors, num_hydrogen_donors, morgan_array, tpsa, j_index, num_rings, num_aromatic_rings, molar_refractivity

# 기존 코드에 수정된 함수로 새로운 특성 추가
train[[
    'logP', 'apka', 'num_rotatable_bonds', 'num_hydrogen_acceptors',
    'num_hydrogen_donors', 'morgan_fingerprint',
    'tpsa', 'j_index', 'num_rings', 'num_aromatic_rings', 'molar_refractivity'
]] = train['SMILES'].apply(calculate_metabolic_stability_descriptors).apply(pd.Series)

test[[
    'logP', 'apka', 'num_rotatable_bonds', 'num_hydrogen_acceptors',
    'num_hydrogen_donors', 'morgan_fingerprint',
    'tpsa', 'j_index', 'num_rings', 'num_aromatic_rings', 'molar_refractivity'
]] = test['SMILES'].apply(calculate_metabolic_stability_descriptors).apply(pd.Series)

train


,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,...,apka,num_rotatable_bonds,num_hydrogen_acceptors,num_hydrogen_donors,morgan_fingerprint,tpsa,j_index,num_rings,num_aromatic_rings,molar_refractivity
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5,2,8,3.259,...,400.504,8,6,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",89.13,1.672477,3,3,109.1199
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2,1,2,2.172,...,301.415,2,4,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",45.23,1.963084,3,2,86.8737
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5,0,3,1.585,...,297.366,3,7,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",62.45,1.741211,4,3,85.0270
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6,0,5,3.475,...,494.665,5,7,0,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",84.22,1.345714,5,3,136.0058
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3,0,1,2.337,...,268.316,1,3,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",42.43,2.022491,3,2,77.1825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3493,TRAIN_3493,Cn1nc(CNC(=O)Cn2nc(C(F)(F)F)c3c2CCC3)c(Cl)c1Cl,1.556,3.079,3.409,396.195,3,1,5,3.409,...,396.200,4,5,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",64.74,1.704243,3,2,84.3287
3494,TRAIN_3494,CCn1[nH]cc/c1=N\C(=O)c1nn(-c2ccccc2)c(=O)c2ccc...,35.560,47.630,1.912,359.381,4,1,3,1.844,...,359.389,3,5,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",85.04,1.976850,4,4,101.6902
3495,TRAIN_3495,CCOC(=O)CCCc1nc2cc(N)ccc2n1C,56.150,1.790,1.941,261.320,3,1,6,2.124,...,261.325,5,5,1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",70.14,2.046226,2,2,74.6974
3496,TRAIN_3496,Nc1cc(C(=O)OCCC2CCOC2=O)cnc1Cl,0.030,2.770,0.989,284.696,5,1,5,0.989,...,284.699,4,6,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",91.51,1.835328,2,1,67.6249


In [4]:
train['AlogP'].fillna(train['AlogP'].median(), inplace=True)
test['AlogP'].fillna(test['AlogP'].median(), inplace=True)

In [6]:
class CustomDataset(Dataset):
    def __init__(self, data, target_col=None, transform=None, is_test=False):
        self.is_test = is_test
        self.transform = transform
        self.is_test = is_test

        if not self.is_test:
            self.data = data.drop(['id', 'SMILES', 'morgan_fingerprint','MLM', 'HLM'], axis=1)
        else: # test
            self.data = data.drop(['id', 'SMILES', 'morgan_fingerprint'], axis=1)


        if self.transform is not None and not self.is_test:  # 훈련 데이터에만 fit_transform 적용
            self.data = self.transform.fit_transform(self.data)
        elif self.transform is not None and self.is_test:  # 테스트 데이터에는 transform만 적용
            self.data = self.transform.transform(self.data)

        if target_col is not None and not self.is_test:
            self.target = data[target_col]

    def __getitem__(self, index):
        features = self.data[index]

        if hasattr(self, 'target'):
            target = self.target[index]
            return torch.tensor(features).to(device).float(), torch.tensor(target).to(device).float().unsqueeze(dim=-1)
        else:
            return torch.tensor(features).to(device).float()

    def __len__(self):
        return len(self.data)


In [7]:
transform = StandardScaler()
transform.fit(train.drop(['id','SMILES','morgan_fingerprint', 'MLM', 'HLM'], axis=1))

train_MLM = CustomDataset(train, target_col='MLM', transform=transform, is_test=False)
train_HLM = CustomDataset(train, target_col='HLM', transform=transform, is_test=False)

In [8]:
X_MLM_train = train_MLM.data
y_MLM_train = train_MLM.target

X_HLM_train = train_HLM.data
y_HLM_train = train_HLM.target

In [9]:
X_MLM_train

array([[ 0.25978464,  0.46045216,  0.65089366, ..., -0.3353936 ,
         0.40083841,  0.53435756],
       [-0.41833617, -0.78569241, -1.33471042, ..., -0.3353936 ,
        -0.50027519, -0.50468403],
       [-0.77668258, -0.83661321,  0.65089366, ...,  0.52662463,
         0.40083841, -0.59093688],
       ...,
       [-0.56018162, -1.28983214, -0.67284239, ..., -1.19741184,
        -0.50027519, -1.07339609],
       [-1.15244861, -0.99585229,  0.65089366, ..., -1.19741184,
        -1.4013888 , -1.40372764],
       [ 0.92048584, -0.86124986, -1.33471042, ..., -0.3353936 ,
         0.40083841, -0.44801507]])

### MLM (XGBoost + NewFeature)

In [49]:
import optuna
from xgboost import XGBRegressor
import numpy as np
from sklearn.model_selection import cross_val_score

# RMSE를 사용하는 스코어 함수 정의
def custom_scorer(y_true, y_pred):
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    return 0.5 * rmse

# Objective 함수 정의
def objective(trial):
    param_space = {
        'nthread': -1,
        'learning_rate': trial.suggest_uniform('learning_rate', 0.0001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1500),
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 15),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.1, 1.0),
        'gamma': trial.suggest_uniform('gamma', 0, 1.0),
        'reg_alpha': trial.suggest_uniform('reg_alpha', 0, 6.0),
        'reg_lambda': trial.suggest_uniform('reg_lambda', 0, 6.0),
    }

    # XGBoost 모델 생성
    model = XGBRegressor(**param_space, random_state=42)
    
    # 교차 검증 수행
    scores = cross_val_score(model, X_MLM_train, y_MLM_train, cv=10, scoring=make_scorer(custom_scorer))
    
    # 교차 검증 스코어의 평균을 사용하여 Objective 함수를 평가
    avg_rmse = np.mean(scores)
    print("Average RMSE:", avg_rmse)
    
    return avg_rmse

# Optuna 스터디 생성
study = optuna.create_study(direction='minimize')  # 최소화 문제로 설정

# 최적의 하이퍼파라미터 탐색
study.optimize(objective, n_trials=100)

# 최적의 하이퍼파라미터와 스코어 출력
best_params = study.best_params
best_score = study.best_value
print("Best Params:", best_params)
print("Best Score:", best_score)

[I 2023-09-13 08:44:10,667] A new study created in memory with name: no-name-0d2d3002-8bac-460c-b9e6-4248d9838425
/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_4460/1090256007.py:15: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_4460/1090256007.py:19: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_4460/1090256007.py:20: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_4460/109025600

Average RMSE: 15.859067297048938


[I 2023-09-13 08:44:43,083] Trial 1 finished with value: 16.737702802287828 and parameters: {'learning_rate': 0.07108950062464742, 'n_estimators': 1164, 'max_depth': 5, 'min_child_weight': 7, 'subsample': 0.5676217849976413, 'colsample_bytree': 0.6441758104602009, 'gamma': 0.19603551288665633, 'reg_alpha': 1.3344712958760012, 'reg_lambda': 5.706400326793586}. Best is trial 0 with value: 15.859067297048938.


Average RMSE: 16.737702802287828


[I 2023-09-13 08:44:47,148] Trial 2 finished with value: 15.790498710146261 and parameters: {'learning_rate': 0.04333564356613575, 'n_estimators': 480, 'max_depth': 3, 'min_child_weight': 11, 'subsample': 0.7371936563463231, 'colsample_bytree': 0.23828492672928867, 'gamma': 0.5993775912630007, 'reg_alpha': 0.26066776231612687, 'reg_lambda': 2.429714864701388}. Best is trial 2 with value: 15.790498710146261.


...

Average RMSE: 15.669082328382428


[I 2023-09-13 09:37:15,248] Trial 298 finished with value: 15.642832206877213 and parameters: {'learning_rate': 0.022216467009900327, 'n_estimators': 492, 'max_depth': 4, 'min_child_weight': 15, 'subsample': 0.5005928754744513, 'colsample_bytree': 0.9425340387977956, 'gamma': 0.6344088298934034, 'reg_alpha': 3.3253379559020813, 'reg_lambda': 0.6289195120385664}. Best is trial 144 with value: 15.61389616555152.


Average RMSE: 15.642832206877213


[I 2023-09-13 09:37:23,238] Trial 299 finished with value: 15.626221128973404 and parameters: {'learning_rate': 0.016995480472794804, 'n_estimators': 442, 'max_depth': 4, 'min_child_weight': 13, 'subsample': 0.532607789610486, 'colsample_bytree': 0.85411947108422, 'gamma': 0.6871759750228064, 'reg_alpha': 3.5626416262617893, 'reg_lambda': 0.4865783253373755}. Best is trial 144 with value: 15.61389616555152.


Average RMSE: 15.626221128973404
Best Params: {'learning_rate': 0.015698920133480516, 'n_estimators': 395, 'max_depth': 4, 'min_child_weight': 15, 'subsample': 0.5121557728100843, 'colsample_bytree': 0.7865528222143898, 'gamma': 0.6193593371049394, 'reg_alpha': 3.475491208819742, 'reg_lambda': 0.3281645431486688}
Best Score: 15.61389616555152


In [50]:
# 하이퍼파라미터별 중요도를 확인할 수 있는 그래프
optuna.visualization.plot_param_importances(study)

In [51]:
# 하이퍼파라미터 최적화 과정을 확인
optuna.visualization.plot_optimization_history(study)

In [62]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# 최적의 하이퍼파라미터를 사용하여 XGBoost 모델 생성
best_MLM_model = XGBRegressor(**best_params, early_stopping_rounds=10)  # 생성자에서 설정

# 데이터 분할 (훈련 세트와 검증 세트)
X_train, X_valid, y_train, y_valid = train_test_split(X_MLM_train, y_MLM_train, test_size=0.3, random_state=42)

# 모델 훈련
best_MLM_model.fit(X_train, y_train, 
                   eval_set=[(X_valid, y_valid)],  # 검증 세트 설정
                   verbose=True)                   # 학습 진행 상황 출력

# early stopping에 의해 중지된 라운드 번호를 출력
print("Best iteration:", best_MLM_model.best_iteration)

# 검증 세트를 사용하여 모델 평가
y_valid_pred = best_MLM_model.predict(X_valid)

# 검증 세트에서 RMSE 또는 다른 평가 지표 계산
rmse = np.sqrt(mean_squared_error(y_valid, y_valid_pred))
print("Validation RMSE:", rmse)


[0]	validation_0-rmse:51.97049
[1]	validation_0-rmse:51.80279
[2]	validation_0-rmse:51.63370
[3]	validation_0-rmse:51.46269
[4]	validation_0-rmse:51.29770
[5]	validation_0-rmse:51.13783
[6]	validation_0-rmse:50.97128
[7]	validation_0-rmse:50.81541
[8]	validation_0-rmse:50.65674
[9]	validation_0-rmse:50.49811
[10]	validation_0-rmse:50.33683
[11]	validation_0-rmse:50.18741
[12]	validation_0-rmse:50.03315
[13]	validation_0-rmse:49.87765
[14]	validation_0-rmse:49.73266
[15]	validation_0-rmse:49.57952
[16]	validation_0-rmse:49.43151
[17]	validation_0-rmse:49.27691
[18]	validation_0-rmse:49.13032
[19]	validation_0-rmse:48.98576
[20]	validation_0-rmse:48.83868
[21]	validation_0-rmse:48.69554
[22]	validation_0-rmse:48.55276
[23]	validation_0-rmse:48.41162
[24]	validation_0-rmse:48.27274
[25]	validation_0-rmse:48.13626
[26]	validation_0-rmse:47.99769
[27]	validation_0-rmse:47.85871
[28]	validation_0-rmse:47.71951
[29]	validation_0-rmse:47.58541
[30]	validation_0-rmse:47.45157
[31]	validation_0-

[252]	validation_0-rmse:34.49134
[253]	validation_0-rmse:34.47478
[254]	validation_0-rmse:34.45794
[255]	validation_0-rmse:34.44114
[256]	validation_0-rmse:34.42553
[257]	validation_0-rmse:34.40916
[258]	validation_0-rmse:34.39444
[259]	validation_0-rmse:34.37694
[260]	validation_0-rmse:34.35901
[261]	validation_0-rmse:34.34693
[262]	validation_0-rmse:34.33206
[263]	validation_0-rmse:34.31807
[264]	validation_0-rmse:34.29944
[265]	validation_0-rmse:34.28782
[266]	validation_0-rmse:34.27225
[267]	validation_0-rmse:34.25968
[268]	validation_0-rmse:34.24450
[269]	validation_0-rmse:34.23290
[270]	validation_0-rmse:34.21716
[271]	validation_0-rmse:34.20376
[272]	validation_0-rmse:34.19167
[273]	validation_0-rmse:34.17641
[274]	validation_0-rmse:34.16187
[275]	validation_0-rmse:34.14912
[276]	validation_0-rmse:34.13654
[277]	validation_0-rmse:34.12223
[278]	validation_0-rmse:34.10831
[279]	validation_0-rmse:34.09244
[280]	validation_0-rmse:34.07663
[281]	validation_0-rmse:34.06352
[282]	vali

[501]	validation_0-rmse:32.79887
[502]	validation_0-rmse:32.79434
[503]	validation_0-rmse:32.79271
[504]	validation_0-rmse:32.79307
[505]	validation_0-rmse:32.79290
[506]	validation_0-rmse:32.78948
[507]	validation_0-rmse:32.78856
[508]	validation_0-rmse:32.78584
[509]	validation_0-rmse:32.78467
[510]	validation_0-rmse:32.78509
[511]	validation_0-rmse:32.78068
[512]	validation_0-rmse:32.78091
[513]	validation_0-rmse:32.78064
[514]	validation_0-rmse:32.77839
[515]	validation_0-rmse:32.77745
[516]	validation_0-rmse:32.77632
[517]	validation_0-rmse:32.77335
[518]	validation_0-rmse:32.77008
[519]	validation_0-rmse:32.76719
[520]	validation_0-rmse:32.76588
[521]	validation_0-rmse:32.76364
[522]	validation_0-rmse:32.76172
[523]	validation_0-rmse:32.75912
[524]	validation_0-rmse:32.75869
[525]	validation_0-rmse:32.75630
[526]	validation_0-rmse:32.75480
[527]	validation_0-rmse:32.75473
[528]	validation_0-rmse:32.75283
[529]	validation_0-rmse:32.75164
[530]	validation_0-rmse:32.75064
[531]	vali

[750]	validation_0-rmse:32.52820
[751]	validation_0-rmse:32.52707
[752]	validation_0-rmse:32.52651
[753]	validation_0-rmse:32.52626
[754]	validation_0-rmse:32.52481
[755]	validation_0-rmse:32.52354
[756]	validation_0-rmse:32.52251
[757]	validation_0-rmse:32.52116
[758]	validation_0-rmse:32.52204
[759]	validation_0-rmse:32.52211
[760]	validation_0-rmse:32.52056
[761]	validation_0-rmse:32.51960
[762]	validation_0-rmse:32.52004
[763]	validation_0-rmse:32.51755
[764]	validation_0-rmse:32.51651
[765]	validation_0-rmse:32.51505
[766]	validation_0-rmse:32.51471
[767]	validation_0-rmse:32.51415
[768]	validation_0-rmse:32.51174
[769]	validation_0-rmse:32.51018
[770]	validation_0-rmse:32.51048
[771]	validation_0-rmse:32.51045
[772]	validation_0-rmse:32.51072
[773]	validation_0-rmse:32.51040
[774]	validation_0-rmse:32.51053
[775]	validation_0-rmse:32.51022
[776]	validation_0-rmse:32.51045
[777]	validation_0-rmse:32.51120
[778]	validation_0-rmse:32.50884
[779]	validation_0-rmse:32.50916
[780]	vali

### HLM (XGBoost+NewFeature)

In [53]:
import optuna
from xgboost import XGBRegressor
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score

# RMSE를 사용하는 스코어 함수 정의
def custom_scorer(y_true, y_pred):
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    return 0.5 * rmse

# Objective 함수 정의
def objective(trial):
    param_space = {
        'nthread': -1,
        'learning_rate': trial.suggest_uniform('learning_rate', 0.0001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1500),
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 15),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.1, 1.0),
        'gamma': trial.suggest_uniform('gamma', 0, 1.0),
        'reg_alpha': trial.suggest_uniform('reg_alpha', 0, 6.0),
        'reg_lambda': trial.suggest_uniform('reg_lambda', 0, 6.0),
    }
    
    # XGBoost 모델 생성
    model = XGBRegressor(**param_space, random_state=42)
    
    # 교차 검증 수행
    scores = cross_val_score(model, X_HLM_train, y_HLM_train, cv=10, scoring=make_scorer(custom_scorer))
    
    # 교차 검증 스코어의 평균을 사용하여 Objective 함수를 평가
    avg_rmse = np.mean(scores)
    print("Average RMSE:", avg_rmse)
    
    return avg_rmse

# Optuna 스터디 생성
study = optuna.create_study(direction='minimize')  # 최소화 문제로 설정

# 최적의 하이퍼파라미터 탐색
study.optimize(objective, n_trials=200)

# 최적의 하이퍼파라미터와 스코어 출력
best_params = study.best_params
best_score = study.best_value
print("Best Params:", best_params)
print("Best Score:", best_score)

[I 2023-09-13 10:11:08,798] A new study created in memory with name: no-name-3b7ecb80-c0eb-443d-827e-1844fbad5cf1
/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_4460/3024585341.py:17: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_4460/3024585341.py:21: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_4460/3024585341.py:22: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_4460/302458534

Average RMSE: 16.741258380889704


[I 2023-09-13 10:11:23,620] Trial 1 finished with value: 16.25982955770581 and parameters: {'learning_rate': 0.06832398375170767, 'n_estimators': 524, 'max_depth': 2, 'min_child_weight': 13, 'subsample': 0.9027788657685127, 'colsample_bytree': 0.44785775928544574, 'gamma': 0.7070388144120371, 'reg_alpha': 5.218704772151714, 'reg_lambda': 4.675008687488688}. Best is trial 1 with value: 16.25982955770581.


Average RMSE: 16.25982955770581


[I 2023-09-13 10:11:25,497] Trial 2 finished with value: 16.279580714575648 and parameters: {'learning_rate': 0.04486498337623788, 'n_estimators': 121, 'max_depth': 6, 'min_child_weight': 6, 'subsample': 0.5107170272377268, 'colsample_bytree': 0.3216547295217321, 'gamma': 0.8651578802700315, 'reg_alpha': 0.5954053181961079, 'reg_lambda': 1.7048489496940535}. Best is trial 1 with value: 16.25982955770581.


...

Average RMSE: 16.142457828259086


[I 2023-09-13 11:25:11,435] Trial 198 finished with value: 17.14635727560947 and parameters: {'learning_rate': 0.002119885650047177, 'n_estimators': 780, 'max_depth': 5, 'min_child_weight': 9, 'subsample': 0.6339312920063443, 'colsample_bytree': 0.6614932154522604, 'gamma': 0.8190708982486262, 'reg_alpha': 3.176143345507771, 'reg_lambda': 3.757554825740069}. Best is trial 185 with value: 16.125754676139852.


Average RMSE: 17.14635727560947


[I 2023-09-13 11:25:35,924] Trial 199 finished with value: 16.957236810729672 and parameters: {'learning_rate': 0.06291100726423447, 'n_estimators': 846, 'max_depth': 5, 'min_child_weight': 10, 'subsample': 0.6231698689808757, 'colsample_bytree': 0.7051416830590204, 'gamma': 0.8516982944742306, 'reg_alpha': 3.4954691626475745, 'reg_lambda': 3.969515369357607}. Best is trial 185 with value: 16.125754676139852.


Average RMSE: 16.957236810729672
Best Params: {'learning_rate': 0.005830624073806664, 'n_estimators': 801, 'max_depth': 5, 'min_child_weight': 10, 'subsample': 0.6327331233861967, 'colsample_bytree': 0.6802107877684083, 'gamma': 0.7970144930903552, 'reg_alpha': 3.217307677617496, 'reg_lambda': 3.6458903066822725}
Best Score: 16.125754676139852


In [56]:
# 하이퍼파라미터별 중요도를 확인할 수 있는 그래프
optuna.visualization.plot_param_importances(study)

In [57]:
# 하이퍼파라미터 최적화 과정을 확인
optuna.visualization.plot_optimization_history(study)

In [58]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# 최적의 하이퍼파라미터를 사용하여 XGBoost 모델 생성
best_HLM_model = XGBRegressor(**best_params, early_stopping_rounds=10)  # 생성자에서 설정

# 데이터 분할 (훈련 세트와 검증 세트)
X_train, X_valid, y_train, y_valid = train_test_split(X_HLM_train, y_HLM_train, test_size=0.2, random_state=42)

# 모델 훈련
best_HLM_model.fit(X_train, y_train, 
                   eval_set=[(X_valid, y_valid)],  # 검증 세트 설정
                   verbose=True)                   # 학습 진행 상황 출력

# early stopping에 의해 중지된 라운드 번호를 출력
print("Best iteration:", best_HLM_model.best_iteration)

# 검증 세트를 사용하여 모델 평가
y_valid_pred = best_HLM_model.predict(X_valid)

# 검증 세트에서 RMSE 또는 다른 평가 지표 계산
rmse = np.sqrt(mean_squared_error(y_valid, y_valid_pred))
print("Validation RMSE:", rmse)


[0]	validation_0-rmse:62.60638
[1]	validation_0-rmse:62.34646
[2]	validation_0-rmse:62.08464
[3]	validation_0-rmse:61.82221
[4]	validation_0-rmse:61.56062
[5]	validation_0-rmse:61.30320
[6]	validation_0-rmse:61.04778
[7]	validation_0-rmse:60.80601
[8]	validation_0-rmse:60.55840
[9]	validation_0-rmse:60.31198
[10]	validation_0-rmse:60.06506
[11]	validation_0-rmse:59.82465
[12]	validation_0-rmse:59.57960
[13]	validation_0-rmse:59.34117
[14]	validation_0-rmse:59.11130
[15]	validation_0-rmse:58.87644
[16]	validation_0-rmse:58.64186
[17]	validation_0-rmse:58.40891
[18]	validation_0-rmse:58.17693
[19]	validation_0-rmse:57.95110
[20]	validation_0-rmse:57.72409
[21]	validation_0-rmse:57.49985
[22]	validation_0-rmse:57.27513
[23]	validation_0-rmse:57.05419
[24]	validation_0-rmse:56.83322
[25]	validation_0-rmse:56.61370
[26]	validation_0-rmse:56.39908
[27]	validation_0-rmse:56.18339
[28]	validation_0-rmse:55.96831
[29]	validation_0-rmse:55.76299
[30]	validation_0-rmse:55.54983
[31]	validation_0-

[252]	validation_0-rmse:34.92510
[253]	validation_0-rmse:34.90050
[254]	validation_0-rmse:34.87506
[255]	validation_0-rmse:34.85083
[256]	validation_0-rmse:34.82508
[257]	validation_0-rmse:34.79733
[258]	validation_0-rmse:34.77556
[259]	validation_0-rmse:34.75308
[260]	validation_0-rmse:34.72813
[261]	validation_0-rmse:34.70354
[262]	validation_0-rmse:34.67980
[263]	validation_0-rmse:34.65449
[264]	validation_0-rmse:34.63373
[265]	validation_0-rmse:34.61207
[266]	validation_0-rmse:34.59139
[267]	validation_0-rmse:34.56886
[268]	validation_0-rmse:34.54916
[269]	validation_0-rmse:34.52954
[270]	validation_0-rmse:34.50859
[271]	validation_0-rmse:34.48593
[272]	validation_0-rmse:34.46599
[273]	validation_0-rmse:34.44667
[274]	validation_0-rmse:34.42575
[275]	validation_0-rmse:34.40380
[276]	validation_0-rmse:34.38198
[277]	validation_0-rmse:34.36358
[278]	validation_0-rmse:34.34169
[279]	validation_0-rmse:34.32092
[280]	validation_0-rmse:34.29844
[281]	validation_0-rmse:34.27878
[282]	vali

[501]	validation_0-rmse:32.73236
[502]	validation_0-rmse:32.73156
[503]	validation_0-rmse:32.73043
[504]	validation_0-rmse:32.73109
[505]	validation_0-rmse:32.73062
[506]	validation_0-rmse:32.72886
[507]	validation_0-rmse:32.72768
[508]	validation_0-rmse:32.72620
[509]	validation_0-rmse:32.72392
[510]	validation_0-rmse:32.72039
[511]	validation_0-rmse:32.72043
[512]	validation_0-rmse:32.71955
[513]	validation_0-rmse:32.71728
[514]	validation_0-rmse:32.71626
[515]	validation_0-rmse:32.71548
[516]	validation_0-rmse:32.71401
[517]	validation_0-rmse:32.71227
[518]	validation_0-rmse:32.70963
[519]	validation_0-rmse:32.70847
[520]	validation_0-rmse:32.70801
[521]	validation_0-rmse:32.70582
[522]	validation_0-rmse:32.70540
[523]	validation_0-rmse:32.70452
[524]	validation_0-rmse:32.70387
[525]	validation_0-rmse:32.70383
[526]	validation_0-rmse:32.70028
[527]	validation_0-rmse:32.69763
[528]	validation_0-rmse:32.69660
[529]	validation_0-rmse:32.69525
[530]	validation_0-rmse:32.69241
[531]	vali

### FingerPrint Model

In [63]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [64]:
import pandas as pd
import numpy as np
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors

def calculate_metabolic_stability_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    logP = Descriptors.MolLogP(mol)
    # 화합물의 친유성을 측정한 것으로 지질 또는 비극성 환경에서의 용해도를 나타냅니다. 생물학적 막을 통과하는 화합물의 능력을 반영합니다.
    apka = Descriptors.MolWt(mol)
    # 화합물의 산 해리 상수의 추정치로 다양한 pH 조건에서 이온화 거동에 대한 정보를 제공합니다.
    num_rotatable_bonds = Descriptors.NumRotatableBonds(mol)
    # 화합물에서 회전 가능한 결합의 수입니다. 이것은 화합물의 유연성과 효소 또는 다른 분자와의 잠재적인 상호 작용에 대한 통찰력을 제공할 수 있습니다.
    num_heteroatoms = Descriptors.NumHeteroatoms(mol)
    # 분자 내 헤테로원자(탄소 및 수소 이외의 원자) 수. 이는 화합물의 반응성과 대사 안정성에 영향을 줄 수 있습니다.
    num_hydrogen_acceptors = Descriptors.NumHAcceptors(mol)
    # 분자 내 수소 결합 수용체의 수. 이들은 결합 및 반응성에 영향을 미치는 다른 분자의 수소 결합 기증자와 상호 작용할 수 있는 사이트입니다.
    num_hydrogen_donors = Descriptors.NumHDonors(mol)
    # 분자 내 수소 결합 기증자의 수입니다. 이들은 수소 결합 상호작용에서 수소 원자를 제공할 수 있는 사이트입니다.
    # morgan_fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
    morgan_fingerprint = AllChem.GetHashedMorganFingerprint(mol, 6, nBits=4096)
    # 분자 하위 구조의 이진 벡터 표현입니다. 이 열에는 화합물과 효소의 상호 작용 및 대사 안정성에 영향을 줄 수 있는 구조적 특징을 포착하는 이진 지문이 포함되어 있습니다.
    morgan_array = np.zeros((1,), dtype=np.int8)
    DataStructs.ConvertToNumpyArray(morgan_fingerprint, morgan_array)

    return logP, apka, num_rotatable_bonds, num_heteroatoms, num_hydrogen_acceptors, num_hydrogen_donors, morgan_array

train[[
    'logP', 'apka', 'num_rotatable_bonds', 'num_heteroatoms',
    'num_hydrogen_acceptors', 'num_hydrogen_donors', 'morgan_fingerprint'
]] = train['SMILES'].apply(calculate_metabolic_stability_descriptors).apply(pd.Series)

test[[
    'logP', 'apka', 'num_rotatable_bonds', 'num_heteroatoms',
    'num_hydrogen_acceptors', 'num_hydrogen_donors', 'morgan_fingerprint'
]] = test['SMILES'].apply(calculate_metabolic_stability_descriptors).apply(pd.Series)

train


,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,logP,apka,num_rotatable_bonds,num_heteroatoms,num_hydrogen_acceptors,num_hydrogen_donors,morgan_fingerprint
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5,2,8,3.259,117.37,3.87744,400.504,8,8,6,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2,1,2,2.172,73.47,3.35474,301.415,2,5,4,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5,0,3,1.585,62.45,1.20450,297.366,3,7,7,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6,0,5,3.475,92.60,3.89356,494.665,5,9,7,0,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3,0,1,2.337,42.43,2.81772,268.316,1,4,3,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3493,TRAIN_3493,Cn1nc(CNC(=O)Cn2nc(C(F)(F)F)c3c2CCC3)c(Cl)c1Cl,1.556,3.079,3.409,396.195,3,1,5,3.409,64.74,2.74730,396.200,4,11,5,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3494,TRAIN_3494,CCn1[nH]cc/c1=N\C(=O)c1nn(-c2ccccc2)c(=O)c2ccc...,35.560,47.630,1.912,359.381,4,1,3,1.844,77.37,2.27630,359.389,3,7,5,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3495,TRAIN_3495,CCOC(=O)CCCc1nc2cc(N)ccc2n1C,56.150,1.790,1.941,261.320,3,1,6,2.124,70.14,2.04130,261.325,5,5,5,1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3496,TRAIN_3496,Nc1cc(C(=O)OCCC2CCOC2=O)cnc1Cl,0.030,2.770,0.989,284.696,5,1,5,0.989,91.51,1.42720,284.699,4,7,6,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."


In [65]:
train['AlogP'].fillna(train['AlogP'].median(), inplace=True)
test['AlogP'].fillna(test['AlogP'].median(), inplace=True)

In [66]:
class CustomDataset(Dataset):
    def __init__(self, data, target_col=None, transform=None, is_test=False):
        self.is_test = is_test
        self.transform = transform
        self.is_test = is_test

        if not self.is_test:
            self.data = self.transform.fit_transform(np.stack(data['morgan_fingerprint']))
        else: # test
            self.data = self.transform.transform(np.stack(data['morgan_fingerprint']))

        if target_col is not None and not self.is_test:
            self.target = data[target_col]

    def __getitem__(self, index):
        features = self.data[index]

        if hasattr(self, 'target'):
            target = self.target[index]
            return torch.tensor(features).to(device).float(), torch.tensor(target).to(device).float().unsqueeze(dim=-1)
        else:
            return torch.tensor(features).to(device).float()

    def __len__(self):
        return len(self.data)


In [67]:
transform = VarianceThreshold(threshold=0.05)

train_MLM = CustomDataset(train, target_col='MLM', transform=transform, is_test=False)
train_HLM = CustomDataset(train, target_col='HLM', transform=transform, is_test=False)

In [68]:
X_MLM_train = train_MLM.data
y_MLM_train = train_MLM.target

X_HLM_train = train_HLM.data
y_HLM_train = train_HLM.target

### MLM (XGBoost+FingerPrint)

In [73]:
import optuna
from xgboost import XGBRegressor
import numpy as np
from sklearn.model_selection import cross_val_score

# RMSE를 사용하는 스코어 함수 정의
def custom_scorer(y_true, y_pred):
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    return 0.5 * rmse

# Objective 함수 정의
def objective(trial):
    param_space = {
        'nthread': -1,
        'learning_rate': trial.suggest_uniform('learning_rate', 0.0001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1500),
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 15),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.1, 1.0),
        'gamma': trial.suggest_uniform('gamma', 0, 1.0),
        'reg_alpha': trial.suggest_uniform('reg_alpha', 0, 6.0),
        'reg_lambda': trial.suggest_uniform('reg_lambda', 0, 6.0),
    }

    # XGBoost 모델 생성
    model = XGBRegressor(**param_space, random_state=42)
    
    # 교차 검증 수행
    scores = cross_val_score(model, X_MLM_train, y_MLM_train, cv=10, scoring=make_scorer(custom_scorer))
    
    # 교차 검증 스코어의 평균을 사용하여 Objective 함수를 평가
    avg_rmse = np.mean(scores)
    print("Average RMSE:", avg_rmse)
    
    return avg_rmse

# Optuna 스터디 생성
study = optuna.create_study(direction='minimize')  # 최소화 문제로 설정

# 최적의 하이퍼파라미터 탐색
study.optimize(objective, n_trials=300)  # 적은 횟수로 탐색

# 최적의 하이퍼파라미터와 스코어 출력
best_params = study.best_params
best_score = study.best_value
print("Best Params:", best_params)
print("Best Score:", best_score)

[I 2023-09-13 16:49:38,176] A new study created in memory with name: no-name-fd80159f-d2f8-425f-b6e6-c8e3497de6c9
/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_4460/3519797691.py:15: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_4460/3519797691.py:19: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_4460/3519797691.py:20: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_4460/351979769

Average RMSE: 16.0476646350136


[I 2023-09-13 16:49:58,805] Trial 1 finished with value: 16.210481608637885 and parameters: {'learning_rate': 0.0359220928633767, 'n_estimators': 296, 'max_depth': 2, 'min_child_weight': 1, 'subsample': 0.8115321321607099, 'colsample_bytree': 0.3071353497408463, 'gamma': 0.29138687943373454, 'reg_alpha': 4.1122716574445, 'reg_lambda': 3.4688604605016735}. Best is trial 0 with value: 16.0476646350136.


Average RMSE: 16.210481608637885


[I 2023-09-13 16:50:31,346] Trial 2 finished with value: 16.10170947013456 and parameters: {'learning_rate': 0.06260130004375927, 'n_estimators': 1070, 'max_depth': 4, 'min_child_weight': 8, 'subsample': 0.7927830534315263, 'colsample_bytree': 0.4017931150882719, 'gamma': 0.9854699300549703, 'reg_alpha': 2.081293306021794, 'reg_lambda': 0.0986717324566524}. Best is trial 0 with value: 16.0476646350136.


...

Average RMSE: 15.699736564814813


[I 2023-09-13 19:01:01,435] Trial 298 finished with value: 15.72030410573581 and parameters: {'learning_rate': 0.011650541149604501, 'n_estimators': 840, 'max_depth': 10, 'min_child_weight': 6, 'subsample': 0.5985219410038914, 'colsample_bytree': 0.5278072435919726, 'gamma': 0.5625323753754077, 'reg_alpha': 3.015963516755368, 'reg_lambda': 2.6491161876716514}. Best is trial 282 with value: 15.671816018618832.


Average RMSE: 15.72030410573581


[I 2023-09-13 19:02:07,447] Trial 299 finished with value: 15.723396893722072 and parameters: {'learning_rate': 0.012668544710873532, 'n_estimators': 931, 'max_depth': 10, 'min_child_weight': 6, 'subsample': 0.6021412414280404, 'colsample_bytree': 0.5274422507063286, 'gamma': 0.6119766419760186, 'reg_alpha': 3.2730766497970434, 'reg_lambda': 2.6127199239056544}. Best is trial 282 with value: 15.671816018618832.


Average RMSE: 15.723396893722072
Best Params: {'learning_rate': 0.013351838203935932, 'n_estimators': 615, 'max_depth': 10, 'min_child_weight': 7, 'subsample': 0.5702815397066843, 'colsample_bytree': 0.3912588837760295, 'gamma': 0.5746059628347633, 'reg_alpha': 3.483239315600075, 'reg_lambda': 2.492600103926944}
Best Score: 15.671816018618832


In [74]:
# 하이퍼파라미터별 중요도를 확인할 수 있는 그래프
optuna.visualization.plot_param_importances(study)

In [75]:
# 하이퍼파라미터 최적화 과정을 확인
optuna.visualization.plot_optimization_history(study)

In [76]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# 최적의 하이퍼파라미터를 사용하여 XGBoost 모델 생성
best_MLM_Fmodel = XGBRegressor(**best_params, early_stopping_rounds=10)  # 생성자에서 설정

# 데이터 분할 (훈련 세트와 검증 세트)
X_train, X_valid, y_train, y_valid = train_test_split(X_MLM_train, y_MLM_train, test_size=0.2, random_state=42)

# 모델 훈련
best_MLM_Fmodel.fit(X_train, y_train, 
                   eval_set=[(X_valid, y_valid)],  # 검증 세트 설정
                   verbose=True)                   # 학습 진행 상황 출력

# early stopping에 의해 중지된 라운드 번호를 출력
print("Best iteration:", best_MLM_Fmodel.best_iteration)

# 검증 세트를 사용하여 모델 평가
y_valid_pred = best_MLM_Fmodel.predict(X_valid)

# 검증 세트에서 RMSE 또는 다른 평가 지표 계산
rmse = np.sqrt(mean_squared_error(y_valid, y_valid_pred))
print("Validation RMSE:", rmse)

[0]	validation_0-rmse:51.78463
[1]	validation_0-rmse:51.41408
[2]	validation_0-rmse:51.06032
[3]	validation_0-rmse:50.71595
[4]	validation_0-rmse:50.38892
[5]	validation_0-rmse:50.04729
[6]	validation_0-rmse:49.70526
[7]	validation_0-rmse:49.40703
[8]	validation_0-rmse:49.08660
[9]	validation_0-rmse:48.77701
[10]	validation_0-rmse:48.48325
[11]	validation_0-rmse:48.18118
[12]	validation_0-rmse:47.90648
[13]	validation_0-rmse:47.61549
[14]	validation_0-rmse:47.31923
[15]	validation_0-rmse:47.04040
[16]	validation_0-rmse:46.77266
[17]	validation_0-rmse:46.49633
[18]	validation_0-rmse:46.22767
[19]	validation_0-rmse:45.98667
[20]	validation_0-rmse:45.73850
[21]	validation_0-rmse:45.50814
[22]	validation_0-rmse:45.27838
[23]	validation_0-rmse:45.03942
[24]	validation_0-rmse:44.80514
[25]	validation_0-rmse:44.58919
[26]	validation_0-rmse:44.36536
[27]	validation_0-rmse:44.15693
[28]	validation_0-rmse:43.94369
[29]	validation_0-rmse:43.72897
[30]	validation_0-rmse:43.51660
[31]	validation_0-

[252]	validation_0-rmse:33.42484
[253]	validation_0-rmse:33.41793
[254]	validation_0-rmse:33.41389
[255]	validation_0-rmse:33.40988
[256]	validation_0-rmse:33.41069
[257]	validation_0-rmse:33.40681
[258]	validation_0-rmse:33.39691
[259]	validation_0-rmse:33.39488
[260]	validation_0-rmse:33.39577
[261]	validation_0-rmse:33.39162
[262]	validation_0-rmse:33.38760
[263]	validation_0-rmse:33.38522
[264]	validation_0-rmse:33.38459
[265]	validation_0-rmse:33.38509
[266]	validation_0-rmse:33.37566
[267]	validation_0-rmse:33.37222
[268]	validation_0-rmse:33.36716
[269]	validation_0-rmse:33.36735
[270]	validation_0-rmse:33.36864
[271]	validation_0-rmse:33.36901
[272]	validation_0-rmse:33.36516
[273]	validation_0-rmse:33.36198
[274]	validation_0-rmse:33.35814
[275]	validation_0-rmse:33.35546
[276]	validation_0-rmse:33.34916
[277]	validation_0-rmse:33.34284
[278]	validation_0-rmse:33.34717
[279]	validation_0-rmse:33.34294
[280]	validation_0-rmse:33.34275
[281]	validation_0-rmse:33.34671
[282]	vali

### HLM (XGBoost+FingerPrint)

In [77]:
import optuna
from xgboost import XGBRegressor
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score

# RMSE를 사용하는 스코어 함수 정의
def custom_scorer(y_true, y_pred):
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    return 0.5 * rmse

# Objective 함수 정의
def objective(trial):
    param_space = {
        'nthread': -1,
        'learning_rate': trial.suggest_uniform('learning_rate', 0.001, 0.1),  # 더 작은 학습률로 시도
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.1, 0.6),
        'n_estimators': trial.suggest_int('n_estimators', 400, 1500),  # 범위 확장
        'gamma': trial.suggest_uniform('gamma', 0, 0.2),
        'reg_alpha': trial.suggest_uniform('reg_alpha', 0, 2.0),
        'reg_lambda': trial.suggest_uniform('reg_lambda', 0, 3.0),
        'scale_pos_weight': trial.suggest_uniform('scale_pos_weight', 1, 10),
        'lambda': trial.suggest_uniform('lambda', 0.2, 0.5),
    }
    
    # XGBoost 모델 생성
    model = XGBRegressor(**param_space, random_state=42)
    
    # 교차 검증 수행
    scores = cross_val_score(model, X_HLM_train, y_HLM_train, cv=10, scoring=make_scorer(custom_scorer))
    
    # 교차 검증 스코어의 평균을 사용하여 Objective 함수를 평가
    avg_rmse = np.mean(scores)
    print("Average RMSE:", avg_rmse)
    
    return avg_rmse

# Optuna 스터디 생성
study = optuna.create_study(direction='minimize')  # 최소화 문제로 설정

# 최적의 하이퍼파라미터 탐색
study.optimize(objective, n_trials=300)

# 최적의 하이퍼파라미터와 스코어 출력
best_params = study.best_params
best_score = study.best_value
print("Best Params:", best_params)
print("Best Score:", best_score)

# 최적의 하이퍼파라미터를 사용하여 XGBoost 모델 생성
best_HLM_Fmodel = XGBRegressor(**best_params)

[I 2023-09-13 19:02:14,892] A new study created in memory with name: no-name-a54aec12-fdd9-46b9-8404-1e25d6d23631
/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_4460/1649836832.py:17: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_4460/1649836832.py:20: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_4460/1649836832.py:21: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_4460/164983683

Average RMSE: 16.336982641466413


[I 2023-09-13 19:03:58,472] Trial 1 finished with value: 17.17790073594181 and parameters: {'learning_rate': 0.09094642419784425, 'max_depth': 8, 'min_child_weight': 5, 'subsample': 0.7674960715828523, 'colsample_bytree': 0.22230592371393557, 'n_estimators': 1390, 'gamma': 0.050021623447861145, 'reg_alpha': 1.463862635704652, 'reg_lambda': 1.8852875479870814, 'scale_pos_weight': 9.281683679034353, 'lambda': 0.2979213158913162}. Best is trial 0 with value: 16.336982641466413.


Average RMSE: 17.17790073594181


[I 2023-09-13 19:04:17,065] Trial 2 finished with value: 16.446228413572452 and parameters: {'learning_rate': 0.012905241832091713, 'max_depth': 9, 'min_child_weight': 1, 'subsample': 0.5801075206675792, 'colsample_bytree': 0.11355781597055128, 'n_estimators': 618, 'gamma': 0.010360444068898623, 'reg_alpha': 1.974692334396886, 'reg_lambda': 1.8794718367544854, 'scale_pos_weight': 7.454332016029561, 'lambda': 0.4302111676117701}. Best is trial 0 with value: 16.336982641466413.


...

Average RMSE: 16.367138351538877


[I 2023-09-14 02:00:51,812] Trial 298 finished with value: 17.170957763439876 and parameters: {'learning_rate': 0.053117589766192415, 'max_depth': 10, 'min_child_weight': 10, 'subsample': 0.6339125340550862, 'colsample_bytree': 0.5254882129107271, 'n_estimators': 1411, 'gamma': 0.0649035831976716, 'reg_alpha': 0.9462146542942997, 'reg_lambda': 1.178966668864794, 'scale_pos_weight': 7.4194137317795175, 'lambda': 0.4138313373140015}. Best is trial 131 with value: 16.22840039072628.


Average RMSE: 17.170957763439876


[I 2023-09-14 02:01:52,720] Trial 299 finished with value: 16.373119117906807 and parameters: {'learning_rate': 0.0035777419575364136, 'max_depth': 7, 'min_child_weight': 10, 'subsample': 0.6502259600920308, 'colsample_bytree': 0.49235586767535977, 'n_estimators': 1303, 'gamma': 0.0699020100899953, 'reg_alpha': 1.0671710044935874, 'reg_lambda': 1.570797232259971, 'scale_pos_weight': 8.435551220164056, 'lambda': 0.49940026371044627}. Best is trial 131 with value: 16.22840039072628.


Average RMSE: 16.373119117906807
Best Params: {'learning_rate': 0.005362229184523802, 'max_depth': 10, 'min_child_weight': 10, 'subsample': 0.6992193907124294, 'colsample_bytree': 0.5312922750535244, 'n_estimators': 1314, 'gamma': 0.1269441173365349, 'reg_alpha': 1.841239779052579, 'reg_lambda': 2.2926268693691316, 'scale_pos_weight': 9.301379905643044, 'lambda': 0.4016381632436306}
Best Score: 16.22840039072628


In [78]:
# 하이퍼파라미터별 중요도를 확인할 수 있는 그래프
optuna.visualization.plot_param_importances(study)

In [79]:
# 하이퍼파라미터 최적화 과정을 확인
optuna.visualization.plot_optimization_history(study)

In [80]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# 최적의 하이퍼파라미터를 사용하여 XGBoost 모델 생성
best_HLM_Fmodel = XGBRegressor(**best_params, early_stopping_rounds=10)  # 생성자에서 설정

# 데이터 분할 (훈련 세트와 검증 세트)
X_train, X_valid, y_train, y_valid = train_test_split(X_HLM_train, y_HLM_train, test_size=0.2, random_state=42)

# 모델 훈련
best_HLM_Fmodel.fit(X_train, y_train, 
                   eval_set=[(X_valid, y_valid)],  # 검증 세트 설정
                   verbose=True)                   # 학습 진행 상황 출력

# early stopping에 의해 중지된 라운드 번호를 출력
print("Best iteration:", best_HLM_Fmodel.best_iteration)

# 검증 세트를 사용하여 모델 평가
y_valid_pred = best_HLM_Fmodel.predict(X_valid)

# 검증 세트에서 RMSE 또는 다른 평가 지표 계산
rmse = np.sqrt(mean_squared_error(y_valid, y_valid_pred))
print("Validation RMSE:", rmse)

[0]	validation_0-rmse:62.64310
[1]	validation_0-rmse:62.40518
[2]	validation_0-rmse:62.17433
[3]	validation_0-rmse:61.94046
[4]	validation_0-rmse:61.71459
[5]	validation_0-rmse:61.48183
[6]	validation_0-rmse:61.25226
[7]	validation_0-rmse:61.03459
[8]	validation_0-rmse:60.81656
[9]	validation_0-rmse:60.59572
[10]	validation_0-rmse:60.37929
[11]	validation_0-rmse:60.16700
[12]	validation_0-rmse:59.95341
[13]	validation_0-rmse:59.73273
[14]	validation_0-rmse:59.53168
[15]	validation_0-rmse:59.31528
[16]	validation_0-rmse:59.09934
[17]	validation_0-rmse:58.89013
[18]	validation_0-rmse:58.68093
[19]	validation_0-rmse:58.47564
[20]	validation_0-rmse:58.27262
[21]	validation_0-rmse:58.06909
[22]	validation_0-rmse:57.86036
[23]	validation_0-rmse:57.65908
[24]	validation_0-rmse:57.45666
[25]	validation_0-rmse:57.25850
[26]	validation_0-rmse:57.06093
[27]	validation_0-rmse:56.87269
[28]	validation_0-rmse:56.67721
[29]	validation_0-rmse:56.48654
[30]	validation_0-rmse:56.28422
[31]	validation_0-

[252]	validation_0-rmse:36.02421
[253]	validation_0-rmse:35.99147
[254]	validation_0-rmse:35.96443
[255]	validation_0-rmse:35.93362
[256]	validation_0-rmse:35.90569
[257]	validation_0-rmse:35.87295
[258]	validation_0-rmse:35.84466
[259]	validation_0-rmse:35.81639
[260]	validation_0-rmse:35.78291
[261]	validation_0-rmse:35.75489
[262]	validation_0-rmse:35.72846
[263]	validation_0-rmse:35.70255
[264]	validation_0-rmse:35.66985
[265]	validation_0-rmse:35.64314
[266]	validation_0-rmse:35.61043
[267]	validation_0-rmse:35.58259
[268]	validation_0-rmse:35.55794
[269]	validation_0-rmse:35.53078
[270]	validation_0-rmse:35.50494
[271]	validation_0-rmse:35.47914
[272]	validation_0-rmse:35.45081
[273]	validation_0-rmse:35.42664
[274]	validation_0-rmse:35.40198
[275]	validation_0-rmse:35.37933
[276]	validation_0-rmse:35.35112
[277]	validation_0-rmse:35.32832
[278]	validation_0-rmse:35.30137
[279]	validation_0-rmse:35.27501
[280]	validation_0-rmse:35.24953
[281]	validation_0-rmse:35.22828
[282]	vali

[501]	validation_0-rmse:32.99087
[502]	validation_0-rmse:32.98835
[503]	validation_0-rmse:32.98604
[504]	validation_0-rmse:32.98287
[505]	validation_0-rmse:32.98115
[506]	validation_0-rmse:32.97847
[507]	validation_0-rmse:32.97678
[508]	validation_0-rmse:32.97281
[509]	validation_0-rmse:32.97010
[510]	validation_0-rmse:32.96936
[511]	validation_0-rmse:32.96373
[512]	validation_0-rmse:32.96318
[513]	validation_0-rmse:32.96084
[514]	validation_0-rmse:32.95940
[515]	validation_0-rmse:32.95548
[516]	validation_0-rmse:32.95079
[517]	validation_0-rmse:32.94870
[518]	validation_0-rmse:32.94657
[519]	validation_0-rmse:32.94300
[520]	validation_0-rmse:32.94213
[521]	validation_0-rmse:32.93814
[522]	validation_0-rmse:32.93538
[523]	validation_0-rmse:32.93260
[524]	validation_0-rmse:32.93122
[525]	validation_0-rmse:32.92783
[526]	validation_0-rmse:32.92244
[527]	validation_0-rmse:32.91895
[528]	validation_0-rmse:32.91727
[529]	validation_0-rmse:32.91302
[530]	validation_0-rmse:32.91000
[531]	vali

### 예측값 구하기

In [82]:
best_MLM_model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.6802107877684083, early_stopping_rounds=10,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.7970144930903552, gpu_id=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.005830624073806664, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=10, missing=nan, monotone_constraints=None,
             n_estimators=801, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [85]:
test_MLM = CustomDataset(test, target_col=None, transform=transform, is_test=True)
test_HLM = CustomDataset(test, target_col=None, transform=transform, is_test=True)

X_MLM_test = test_MLM.data
X_HLM_test = test_HLM.data

# 최적의 에폭에서 중단된 모델을 사용하여 예측 수행
#y_MLM_pred = best_MLM_model.predict(X_MLM_test)
#y_HLM_pred = best_HLM_model.predict(X_HLM_test)

y_MLM_Fpred = best_MLM_Fmodel.predict(X_MLM_test)
y_HLM_Fpred = best_HLM_Fmodel.predict(X_HLM_test)

#ensemble_MLM_pred = (0.8 * y_MLM_pred + 0.2 * y_MLM_Fpred)
#ensemble_HLM_pred = (0.8 * y_HLM_pred + 0.2 * y_HLM_Fpred)

In [86]:
submission = pd.read_csv('./data/sample_submission.csv')

submission['MLM'] = y_MLM_Fpred
submission['HLM'] = y_HLM_Fpred
submission

,id,MLM,HLM
0,TEST_000,31.327168,50.049099
1,TEST_001,54.334167,68.257355
2,TEST_002,36.148605,57.819660
3,TEST_003,29.803299,51.357929
4,TEST_004,44.775391,60.352108
...,...,...,...
478,TEST_478,36.010696,57.647057
479,TEST_479,58.576653,78.157288
480,TEST_480,25.193075,43.275013
481,TEST_481,54.847813,67.197968


In [87]:
submission.to_csv('./submissions/Optuna+XGBoost_submission_6.csv', index=False)